In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [10]:
#week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
week_day_Apr_list = [2, 9, 16, 23, 30]

In [11]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['PM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Apr_list)

14400

In [13]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 600)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 600)

In [15]:
x[:,:2]

matrix([[  5579.79525307,   5714.48936252],
        [  2788.40374704,   2466.24992324],
        [  3110.6419456 ,   2972.17612752],
        [  4860.72284868,   4580.2505983 ],
        [  5846.4814216 ,   4776.85702352],
        [  4317.78815501,   3507.40177929],
        [  2414.55979554,   2050.61381169],
        [  3156.99053784,   2614.47957377],
        [  6213.66574265,   5833.24800427],
        [  5937.07944245,   8851.0659437 ],
        [  8574.49463147,  11551.74482126],
        [  6795.06794891,   5095.35408522],
        [  2152.57945447,   2924.35711844],
        [  3878.6477567 ,   3558.87849197],
        [  2097.20625591,    970.5145146 ],
        [  6513.65669111,   6662.20914609],
        [  2174.85810621,   1717.51178656],
        [  1141.25407058,   1911.47469032],
        [  4928.00211585,   4171.80197284],
        [  6039.64819484,   7489.91024408],
        [  7336.11485544,   6130.86728853],
        [  8840.23376351,   8943.17440903],
        [  3982.83099134,   3102

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [4e-02, 9e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.01s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -3.58929065e+06  0.00000000e+00  6.19e+03 2.06e+02  9.17e+05     0s
   1   2.77190494e+06 -3.84286684e+06  9.92e+01 1.14e-13  1.32e+05     0s
   2   3.04605545e+05 -7.70411780e+05  9.92e-05 5.68e-14  1.92e+04     0s
   3  -4.93738609e+04 -1.95932304e+05  9.92e-11 5.68e-14  2.6

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_PM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[2.7877049398915935e-08,
 3657.289817059476,
 3.059973721967415e-08,
 762.7733821652765,
 1955.1835437376312,
 6.50922774447474e-05,
 8.013348507293573e-06,
 1222.0718088319502,
 1002.7772808032807,
 2981.859079137279,
 526.7008858429685,
 859.0827862889792,
 6.770374657388747e-05,
 7.130885343085633e-05,
 4722.906567104572,
 1456.7122167824716,
 512.2559522111544,
 1119.538390065134,
 4979.420215670909,
 4822.59508856894,
 6.55750929841022e-05,
 7.161248251496416e-05,
 2919.167867333784,
 6.304256029940412e-06,
 2216.269225370803,
 101.58132732074421,
 3.6383302410063103e-05,
 1882.1833642424874,
 6.330349758297711e-06,
 6.3118923635790275e-06,
 6.369557276800003e-06,
 2104.1331814284667,
 3858.7607115171945,
 3.600369520920948e-05,
 3.595929418634153e-05,
 321.92022443470535,
 148.78194765654735,
 0.0003458486687599044,
 2044.6351575534381,
 3270.978037665124,
 3103.7312064998832,
 970.7249316000288,
 1.1001928786299998e-05,
 1.1016462599772105e-05,
 10061.630235200066,
 334.51693574